In [1]:
# to run our back-analyses model, we need to import necessary python module and two .py files, please put the .py file in the same folder of this notebook
import numpy as np 
import pandas as pd 
from co_seismic_simulation import co_landslide_strength_inversion # the module to back-analyses for coseismic phase
from post_seismic_simulation import po_landslide_strength_inversion # the module to back-analyses for post-seismic phase

In [2]:
# then we input our dataset, each row represent a specific post-seismic landslide location.
data=pd.read_csv ("wenchuan_all_data.csv",sep=",",encoding="utf-8",header=0)
# in there, we use first 5 landslides to show a process that using back-analyses model 
data=data.iloc[:5]
print(data)

   peak_ground_acceleration  density  internal_friction_angle  rock_cohesion  \
0                      0.90     26.5                       44             40   
1                      0.92     26.5                       44             40   
2                      0.86     26.2                       42             39   
3                      0.90     26.5                       44             40   
4                      0.88     26.5                       44             40   

       slope  m_before   m_after  time  
0  33.004700         0  0.737017  2011  
1  30.135099         0  0.737017  2011  
2  40.856998         0  0.804696  2011  
3  45.556400         0  0.833959  2011  
4  48.797100         0  0.833959  2011  


In [3]:
# we use code below to calculate the shear strenght 
def calculate_shear_strength(weigth,slope,cohesion,firction,t,m):
    normal_stress=(weigth*t-9.8*t*m)*np.cos(slope*np.pi/180.)
    shear_strength=normal_stress*np.tan(firction*np.pi/180.)+cohesion
    return shear_strength

In [4]:
# calculate the shear strenghth before earthquake
# the module has four input parameters: the input data, thickness, cov value for cohesion and cov for friction angle.
# In there, we need to assign a cov value for cohesion and friction angle, and also a thickness for landslide
# we show a case for thickness equal to 5m where the cov of cohesion is 1.55, cov of friction angle is 0.15
# Note the Cohesion1,friction1 is the limit condition result (e.g FS=1) of hillslope stable and failed, which just a try and we don't use in our manuscript 
# so,the Cohesion1,friction1 is meanless but can't remove because the module need four variables to recieve output. 
lab=co_landslide_strength_inversion(data,cov_c=1.55,tick=5,cov_i_f=0.15)
Cohesion,friction,Cohesion1,friction1=lab.calculate_inversion_strength()# run the model and recieve the output
Cohesion[Cohesion< 0] = np.nan #  the model run 1,000 Monte-Carlo simulations,so "Cohesion" has 1000 values and we remove the failed one    
friction[friction< 0] = np.nan 
Cohesion=np.nanmean(Cohesion,axis=1)# the final result is the mean of 1,000 simulations (see method in paper).
friction=np.nanmean(friction,axis=1)
# calculate the shear strength and remeber change t value
shear_before=calculate_shear_strength(data["density"],data["slope"],Cohesion,friction,t=5,m=data["m_before"])

In [5]:
# show the shear strength of coseismic phase
shear_before

0    229.885497
1    228.865695
2    228.904788
3    242.706501
4    243.601335
dtype: float64

In [6]:
# calculate the shear strenghth for post-seismic phase 
# same to above, we need to assign thickness and cov value  
lab1=po_landslide_strength_inversion(data,cov_c=1.55,tick=5,cov_i_f=0.15)
Cohesion,friction,Cohesion1,friction1=lab1.calculate_inversion_strength()
Cohesion[Cohesion < 0] = np.nan
friction[friction < 0] = np.nan
Cohesion=np.nanmean(Cohesion,axis=1)
friction=np.nanmean(friction,axis=1)
# remeber change t value and m that represent post-seismic phase, this is a different value with m_before
shear_after=calculate_shear_strength(data["density"],data["slope"],Cohesion,friction,t=5,m=data["m_after"])

In [7]:
# show the shear strength of post-seismic phase
shear_after

0    63.820224
1    59.675991
2    70.324890
3    75.230712
4    76.817635
dtype: float64

In [8]:
# calculate RSS value 
reduction_value=shear_before-shear_after
reduction_value

0    166.065273
1    169.189704
2    158.579897
3    167.475790
4    166.783700
dtype: float64

In [9]:
# calculate RSS %
reduction=reduction_value/shear_before
reduction

0    0.722383
1    0.739253
2    0.692777
3    0.690034
4    0.684658
dtype: float64

In [10]:
# the output is .csv file
output=pd.DataFrame({"shear_before":shear_before,"shear_after":shear_after,"reduction":reduction,"reduction_value":reduction_value})
output.to_csv('case of back analyses model.csv')